In [2]:
from datasets import load_dataset, load_from_disk

ds = load_dataset("R5dwMg/zh-wiki-yue-long")

/Users/yorky/PycharmProjects/sinitic-nlu/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds['train'][0]

{'text': '少年阿寶·尼爾，係自護軍攻擊佢居住嘅宇宙都市嗰時，為咗保衞朋友而上咗聯邦第一架實戰用MS「高達」而捲入咗戰爭，故事就係以阿寶為中心，描佢跟住高達嘅母艦白色基地，經歷種種困難而成長嘅故事，重被認定為少見嘅「新人類」。'}

In [2]:
from transformers import BertTokenizerFast, BertModel, BertForMaskedLM
model_name = "hon9kon9ize/bert-base-cantonese"
# model_name = "./canto-pretrain/checkpoint-27000"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
model.save_pretrained('models/bert-base-cantonese')
tokenizer.save_pretrained('models/bert-base-cantonese')

('models/bert-base-cantonese/tokenizer_config.json',
 'models/bert-base-cantonese/special_tokens_map.json',
 'models/bert-base-cantonese/vocab.txt',
 'models/bert-base-cantonese/added_tokens.json',
 'models/bert-base-cantonese/tokenizer.json')

In [11]:
BertTokenizerFast.from_pretrained('hon9kon9ize/bert-base-cantonese').save_pretrained("./hon9kon9ize-canto")

('./hon9kon9ize-canto/tokenizer_config.json',
 './hon9kon9ize-canto/special_tokens_map.json',
 './hon9kon9ize-canto/vocab.txt',
 './hon9kon9ize-canto/added_tokens.json',
 './hon9kon9ize-canto/tokenizer.json')

In [29]:
import torch
text = '[MASK]學校，老師俾咗份卷我。'
# text = '拔萃有個大仆街。' #拔萃有個大仆街。
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
mask_token_index = (inputs["input_ids"] == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
top_k = 5
for i in range(mask_token_index.shape[0]):
    mask_logits = logits[0, mask_token_index[i], :]
    top_tokens = torch.topk(mask_logits, top_k, dim=0).indices.tolist()
    for token_id in top_tokens:
        token = tokenizer.decode([token_id])
        print(f"Predicted token: {token}")

Predicted token: 到
Predicted token: 返
Predicted token: 去
Predicted token: 回
Predicted token: 在


In [4]:
tokenizer.tokenize('然而，Enciclopedia Libre嘅用戶唔排除喺未來重新合併嘅可能性，同埋希望繼續同維基百科保持聯繫。呢場紛爭亦引起咗關於非英文維基百科版本嘅角色嘅廣泛討論，並且直接導致非英文維基百科嘅幾項重大改革。')

['然',
 '而',
 '，',
 '[UNK]',
 '[UNK]',
 '嘅',
 '用',
 '戶',
 '唔',
 '排',
 '除',
 '喺',
 '未',
 '來',
 '重',
 '新',
 '合',
 '併',
 '嘅',
 '可',
 '能',
 '性',
 '，',
 '同',
 '埋',
 '希',
 '望',
 '繼',
 '續',
 '同',
 '維',
 '基',
 '百',
 '科',
 '保',
 '持',
 '聯',
 '繫',
 '。',
 '呢',
 '場',
 '紛',
 '爭',
 '亦',
 '引',
 '起',
 '咗',
 '關',
 '於',
 '非',
 '英',
 '文',
 '維',
 '基',
 '百',
 '科',
 '版',
 '本',
 '嘅',
 '角',
 '色',
 '嘅',
 '廣',
 '泛',
 '討',
 '論',
 '，',
 '並',
 '且',
 '直',
 '接',
 '導',
 '致',
 '非',
 '英',
 '文',
 '維',
 '基',
 '百',
 '科',
 '嘅',
 '幾',
 '項',
 '重',
 '大',
 '改',
 '革',
 '。']

In [8]:
def get_training_corpus():
    return (
        ds['train'][i : i + 1000]['text']
        for i in range(0, len(ds['train']), 1000)
    )
training_corpus = get_training_corpus()

In [9]:
new_tokenizer = tokenizer.train_new_from_iterator(training_corpus, vocab_size=30000)

In [10]:
new_tokenizer.save_pretrained("./canto-pretrain-tokenizer")

('./canto-pretrain-tokenizer/tokenizer_config.json',
 './canto-pretrain-tokenizer/special_tokens_map.json',
 './canto-pretrain-tokenizer/vocab.txt',
 './canto-pretrain-tokenizer/added_tokens.json',
 './canto-pretrain-tokenizer/tokenizer.json')

In [14]:
ds = load_from_disk('data/nlptea_dataset')['train']

In [15]:
example = ds[0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 '仍',
 '記',
 '得',
 '小',
 '學',
 '下',
 '課',
 '的',
 '時',
 '候',
 '，',
 '我',
 '總',
 '愛',
 '跟',
 '表',
 '弟',
 '到',
 '草',
 '推',
 '裏',
 '捉',
 '蠶',
 '蟲',
 '，',
 '每',
 '當',
 '打',
 '開',
 '葉',
 '子',
 '時',
 '看',
 '到',
 '蟲',
 '子',
 '，',
 '心',
 '中',
 '總',
 '有',
 '無',
 '限',
 '的',
 '喜',
 '悅',
 '。',
 '[SEP]']

In [16]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"cantonese_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [17]:
tokenized_ds = ds.map(tokenize_and_align_labels, batched=True)

Map: 100%|██████████| 2062/2062 [00:00<00:00, 11934.36 examples/s]


In [3]:
from simalign import SentenceAligner
myaligner = SentenceAligner(model="bert-base-chinese", matching_methods="m", device="mps")
src_sentence = ["全","部","同","我","企","起","身"]
trg_sentence = ["全","部","跟","我","站","起","来"]
alignments = myaligner.get_word_aligns(src_sentence, trg_sentence)

for matching_method in alignments:
    print(matching_method, ":", alignments[matching_method])

2025-08-21 05:07:16,674 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-chinese


mwmf : [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6)]
